# Solar Statistics Calculations

A number of properties are dependent on solar position etc. We use PVLib to calculate solar position and subsequent properties

In [7]:
import sys
sys.path.append('..')
for p in sys.path:
    print(p)    
import gnomy.constants, gnomy.core, gnomy.utils

c:\Users\liama\OneDrive\NewVSCode\gnomy\notebooks
C:\Users\liama\.config\herbie
C:\Program Files\PerkinElmerInformatics\ChemOffice2020\ChemScript\Lib
c:\Users\liama\miniconda3\envs\lonestar\python311.zip
c:\Users\liama\miniconda3\envs\lonestar\DLLs
c:\Users\liama\miniconda3\envs\lonestar\Lib
c:\Users\liama\miniconda3\envs\lonestar

c:\Users\liama\miniconda3\envs\lonestar\Lib\site-packages
c:\Users\liama\miniconda3\envs\lonestar\Lib\site-packages\win32
c:\Users\liama\miniconda3\envs\lonestar\Lib\site-packages\win32\lib
c:\Users\liama\miniconda3\envs\lonestar\Lib\site-packages\Pythonwin
..
..
..


In [8]:
import pandas as pd
import numpy as np
from pvlib import solarposition
import datetime

The function accepts a datetime index

In [9]:
start_date = datetime.datetime(2020, 1, 1)
end_date = datetime.datetime(2020, 12, 31, 23)
latitude=29.25
longitude=360-98.31
dt_index = pd.date_range(start=start_date, end=end_date, freq='1H')
dt_index

DatetimeIndex(['2020-01-01 00:00:00', '2020-01-01 01:00:00',
               '2020-01-01 02:00:00', '2020-01-01 03:00:00',
               '2020-01-01 04:00:00', '2020-01-01 05:00:00',
               '2020-01-01 06:00:00', '2020-01-01 07:00:00',
               '2020-01-01 08:00:00', '2020-01-01 09:00:00',
               ...
               '2020-12-31 14:00:00', '2020-12-31 15:00:00',
               '2020-12-31 16:00:00', '2020-12-31 17:00:00',
               '2020-12-31 18:00:00', '2020-12-31 19:00:00',
               '2020-12-31 20:00:00', '2020-12-31 21:00:00',
               '2020-12-31 22:00:00', '2020-12-31 23:00:00'],
              dtype='datetime64[ns]', length=8784, freq='H')

We resample to 5 minute measurements so that average measurements are more accurate with nonlinear functions dependent on angles, etc.  

Since the measurements are sometimes aggregated for the whole period previous, we change the range to ($t_{0-1}$ to $t_f$)

In [10]:
offset_timedelta = pd.Timedelta(dt_index.freq)
dt_index_resampled = pd.date_range(start_date - offset_timedelta, end_date, freq='5Min')
dt_index_resampled

DatetimeIndex(['2020-01-01 00:00:00', '2020-01-01 00:05:00',
               '2020-01-01 00:10:00', '2020-01-01 00:15:00',
               '2020-01-01 00:20:00', '2020-01-01 00:25:00',
               '2020-01-01 00:30:00', '2020-01-01 00:35:00',
               '2020-01-01 00:40:00', '2020-01-01 00:45:00',
               ...
               '2020-12-31 22:15:00', '2020-12-31 22:20:00',
               '2020-12-31 22:25:00', '2020-12-31 22:30:00',
               '2020-12-31 22:35:00', '2020-12-31 22:40:00',
               '2020-12-31 22:45:00', '2020-12-31 22:50:00',
               '2020-12-31 22:55:00', '2020-12-31 23:00:00'],
              dtype='datetime64[ns]', length=105397, freq='5T')

Using PVLib, we calculate the `solar zenith angle` and get the average angle for the preceeding period. We also ensure that we have the same start and end dates as the original series. The `label` argument in `df.resample` determines whether an interval is labeled by the start or end of the interval

In [11]:
sp = solarposition.get_solarposition(dt_index_resampled, latitude, longitude)
zenith = sp["zenith"].resample(offset_timedelta, label="right").mean()[start_date:end_date]
zenith

2020-01-01 01:00:00     99.257037
2020-01-01 02:00:00    111.672379
2020-01-01 03:00:00    124.449457
2020-01-01 04:00:00    137.442827
2020-01-01 05:00:00    150.504021
                          ...    
2020-12-31 19:00:00     52.479484
2020-12-31 20:00:00     53.834893
2020-12-31 21:00:00     58.794862
2020-12-31 22:00:00     66.563672
2020-12-31 23:00:00     76.284876
Freq: H, Name: zenith, Length: 8783, dtype: float64